# CRAB Spark tape recall history

This jobs is querying `rules_history` table of cmsrucio to answer theses questions:
- How long do tasks stay in “taperecall”?

In [ ]:
from datetime import datetime, timedelta, timezone
import os
import time
import pandas as pd

from pyspark import SparkContext, StorageLevel
from pyspark.sql import SparkSession
from pyspark.sql.functions import (
    current_user,
    col, collect_list, concat_ws, greatest, lit, lower, when,
    avg as _avg,
    count as _count,
    hex as _hex,
    max as _max,
    min as _min,
    round as _round,
    sum as _sum,
)
from pyspark.sql.types import (
    StructType,
    LongType,
    StringType,
    StructField,
    DoubleType,
    IntegerType,
)

In [ ]:
spark = SparkSession\
        .builder\
        .appName('tape-recall-history')\
        .getOrCreate()
spark

In [ ]:
spark.catalog.clearCache()

In [ ]:
# arguments
# secret path, also check if file exists
secretpath = os.environ.get('OPENSEARCH_SECRET_PATH', f'{os.getcwd()}/../workdir/secret_opensearch.txt')
if not os.path.isfile(secretpath): 
    raise Exception(f'OS secrets file {secretpath} does not exists')
# if PROD, index prefix will be `crab-*`, otherwise `crab-test-*`
PROD = os.environ.get('PROD', 'false').lower() in ('true', '1', 't')
# FROM_DATE, in strptime("%Y-%m-%d")
START = os.environ.get('START_DATE', None) 
END = os.environ.get('END_DATE', None)

In [ ]:
# try to import osearch from current directory, fallback to $PWD/../workdir if not found
try:
    import osearch
except ModuleNotFoundError:
    import sys
    sys.path.insert(0, f'{os.getcwd()}/../workdir')
    import osearch

In [ ]:
# variables for run inside notebook
START_DATE = "2020-01-01"
END_DATE = "2024-10-01"
# if cronjob, replace constant with value from env
if START and END:
    START_DATE = START
    END_DATE = END

In [ ]:
# index name
index_name = 'tape-recall-history' # always put test index prefix
# use prod index pattern if this execution is for production
if PROD:
    index_name = f'crab-prod-{index_name}'
else:
    index_name = f'crab-test-{index_name}'

In [ ]:
# datetime object
start_datetime = datetime.strptime(START_DATE, "%Y-%m-%d").replace(tzinfo=timezone.utc)
end_datetime = datetime.strptime(END_DATE, "%Y-%m-%d").replace(tzinfo=timezone.utc)
# sanity check
if end_datetime < start_datetime: 
    raise Exception(f"end date ({END_DATE}) is less than start date ({START_DATE})")
start_epochmilis = int(start_datetime.timestamp()) * 1000
end_epochmilis = int(end_datetime.timestamp()) * 1000
yesterday_epoch = int((end_datetime-timedelta(days=1)).timestamp())

In [ ]:
# debug
print(START_DATE, 
      END_DATE, 
      index_name,
      sep='\n')

In [ ]:
# Import data into spark

HDFS_RUCIO_RULES_HISTORY = f'/project/awg/cms/rucio/{END_DATE}/rules_history/'

print("==============================================="
      , "RUCIO : Rules History"
      , "==============================================="
      , "File Directory:", HDFS_RUCIO_RULES_HISTORY
      , "Work Directory:", os.getcwd()
      , "==============================================="
      , "===============================================", sep='\n')

# we only interest in the rules where state does not change anymore.
# which means, only the rules that already expired.
rucio_rules_history = (
    spark.read.format('avro').load(HDFS_RUCIO_RULES_HISTORY).withColumn('ID', lower(_hex(col('ID'))))
         .select("ID", "ACCOUNT", "NAME", "STATE", "EXPIRES_AT", "UPDATED_AT", "CREATED_AT")
         .filter(f"""\
                  1=1
                  AND ACTIVITY = 'Analysis TapeRecall'
                  AND EXPIRES_AT >= {start_epochmilis}
                  AND EXPIRES_AT < {end_epochmilis}
                  """)
         .cache()
)
rucio_rules_history.createOrReplaceTempView("rules_history")

HDFS_CRAB_part = f'/project/awg/cms/crab/tasks/{END_DATE}/'
print("==============================================="
      , "CRAB Table"
      , "==============================================="
      , "File Directory:", HDFS_CRAB_part
      , "Work Directory:", os.getcwd()
      , "==============================================="
      , "===============================================", sep='\n')

# do not filter taskdb by create time (TM_START_TIME) because it is possible that rules are created 6 months ago
tasks_df = (
    spark.read.format('avro').load(HDFS_CRAB_part)
         .select("TM_TASKNAME","TM_START_TIME","TM_TASK_STATUS",  'TM_TASKNAME', 'TM_START_TIME', 'TM_TASK_STATUS' , 'TM_DDM_REQID')
         .cache()
)
tasks_df.createOrReplaceTempView("tasks")

In [ ]:
# rucio append new row to rules_history when the content rules table change (not sure the exact condition)
# We need to get "the latest" row for each rules by:
# - If rule has state "O", select the earliest UPDATED_AT row.
#   For the OK rule, we can calculate number of days using UPDATED_AT-CREATED_AT. 
#   However, there are some posiblility that rucio append new entry with newer UPDATED_AT (For exmple 37fcada73f14439b88558ef792e10276)
# - If not, select the latest UPDATED_AT row.
#   This because the rules still in temporary state, and the rules will go to the end state 
#   (not the real state, but rules_history will not getting new row anymore) after rules is expired 
#   So, we can calculate number of day by EXPIRES_AT-CREATED_AT
#
# Here is the step to translate above condition to SQL (in the buttom-up manner)
# 1. count number of row where the state is 'O'.
# 2. left join the rule history by ID, so each row will have number of state O 
#    New table look like this:
#    +--------------------------------+-----+-------------+-------+
#    |ID                              |STATE|EXPIRES_AT   |state_o|
#    +--------------------------------+-----+-------------+-------+
#    |6d275222b43d431abc568dd83313118f|R    |1727244523000|1      |
#    |875a388ca374407ea761689511078956|R    |1727339056000|1      |
#    |dfe4012bcb9c448f98f940f01302ae6e|R    |1727234937000|0      |
#    |dfe4012bcb9c448f98f940f01302ae6e|R    |1725402537000|0      |
#    |c6859b18a771440ab906733e2bebf78a|R    |1727235038000|1      |
#     
# 3. select the earliest row for "the rule that have state O" (where clause). this can be done by windows function, sort by UPDATED_AT ascending for each ID, then filter only row_number "1"
# 4. select the latest row for "the rule that does not have state O at all". 
#    This is a bit tricky but can be done by filter out the rule that have number of state O more than zero.
#    which this column already availabe from left join in step 2.
#    For the "select latest row" we do the same way as 4. but sort by UPDATED_AT descending instead.
# 5. merge result from 3. and 4 with UNION ALL.
# 6. Then, we will calculate number of date in the next step
#
# We are selecting the rules for each condition and join later, to avoid large broadcasthashjoin internally
# I (Wa) tried this before and it cause above issue, but I might be wrong here though.
# ```
#  SELECT * FROM rhistinfo_t 
#  WHERE (state_o > 0) 
#    OR (ID NOT IN (SELECT ID FROM (SELECT * FROM rhistinfo_t WHERE state_o > 0)))
# ```
# 

query = f"""\
WITH 
count_t AS (
SELECT ID, 
       SUM(CASE WHEN state = 'O' THEN 1 ELSE 0 END) AS state_o
FROM rules_history
GROUP BY ID
),
rhistinfo_t AS (
SELECT rules_history.ID AS ID, 
       rules_history.ACCOUNT AS ACCOUNT, 
       rules_history.NAME AS NAME, 
       rules_history.STATE AS STATE, 
       rules_history.EXPIRES_AT AS EXPIRES_AT, 
       rules_history.UPDATED_AT AS UPDATED_AT, 
       rules_history.CREATED_AT AS CREATED_AT,
       count_t.state_o AS state_o
FROM rules_history
LEFT JOIN count_t ON rules_history.ID = count_t.ID
),
tmpwindow_1 AS (
SELECT *, row_number() over(partition by ID order by UPDATED_AT) as row_num
FROM rhistinfo_t
WHERE STATE = 'O'
), 
r1 AS (
SELECT * FROM tmpwindow_1
WHERE row_num = 1
),
tmpwindow_2 AS (
SELECT *, row_number() over(partition by ID order by UPDATED_AT DESC) as row_num
FROM rhistinfo_t
WHERE STATE != 'O' AND state_o = 0
),
r2 AS (
SELECT * FROM tmpwindow_2
WHERE row_num = 1
),
r_all AS (
SELECT * FROM r1
UNION ALL
SELECT * FROM r2
)
SELECT * 
FROM r_all
ORDER BY ID
"""

tmprules = spark.sql(query)
tmprules.show(10, False)
tmprules.createOrReplaceTempView("tmprules")

In [ ]:
# Calculate number of days, for state O, UPDATED_AT-CREATED_AT, otherwise EXPIRES_AT-CREATED_AT
# then enrich the data with the crab taskdb table by join rule ID with TM_DDM_REQID column
# need to apply windows function again to select only the rule id with the latest crab tasks

query = f"""\
WITH 
calc_days_t AS (
SELECT ID, ACCOUNT, NAME, STATE, EXPIRES_AT, UPDATED_AT, CREATED_AT,
       CASE 
           WHEN STATE = 'O' THEN ceil((UPDATED_AT-CREATED_AT)/86400000)  
           ELSE ceil((EXPIRES_AT-CREATED_AT)/86400000)
       END AS DAYS
FROM tmprules
),
join_t AS (
SELECT 
    calc_days_t.ID AS ID, 
    calc_days_t.ACCOUNT AS ACCOUNT, 
    calc_days_t.NAME AS NAME, 
    calc_days_t.STATE AS STATE, 
    calc_days_t.DAYS AS DAYS, 
    calc_days_t.EXPIRES_AT AS EXPIRES_AT, 
    calc_days_t.UPDATED_AT AS UPDATED_AT, 
    calc_days_t.CREATED_AT AS CREATED_AT, 
    tasks.TM_TASKNAME AS TM_TASKNAME,
    IFNULL(tasks.TM_START_TIME, 0) AS TM_START_TIME, 
    tasks.TM_TASK_STATUS AS TM_TASK_STATUS
FROM calc_days_t
LEFT JOIN tasks ON calc_days_t.ID = tasks.TM_DDM_REQID
),
window_t AS (
SELECT ID, ACCOUNT, NAME, STATE, DAYS, EXPIRES_AT, UPDATED_AT, CREATED_AT, TM_TASKNAME, TM_START_TIME, TM_TASK_STATUS, 
       row_number() OVER (PARTITION BY ID ORDER BY TM_START_TIME DESC) AS row_num
FROM join_t 
),
uniqueid_t AS (
SELECT *
FROM window_t 
WHERE row_num = 1
), 
finalize_t AS (
SELECT ID, ACCOUNT, NAME, STATE, DAYS, EXPIRES_AT, UPDATED_AT, CREATED_AT, TM_TASKNAME, IFNULL(TM_START_TIME, 0) as TM_START_TIME, TM_TASK_STATUS, 
       EXPIRES_AT AS timestamp,
       'tape_recall_history' AS type
FROM uniqueid_t 
)
SELECT *
FROM finalize_t
"""

tmpdf = spark.sql(query)
tmpdf.show(10, False)


In [ ]:
tmpdf.count()

In [ ]:
docs = tmpdf.toPandas().to_dict('records')

In [ ]:
schema = {
        "settings": {"index": {"number_of_shards": "1", "number_of_replicas": "1"}},
        "mappings": {
            "properties": {
                "ID": {"ignore_above": 2048, "type": "keyword"},
                "ACCOUNT": {"ignore_above": 2048, "type": "keyword"},
                "NAME": {"ignore_above": 2048, "type": "keyword"},
                "STATE": {"ignore_above": 2048, "type": "keyword"},
                "DAYS": {"type": "long"},
                "EXPIRES_AT": {"format": "epoch_millis", "type": "date"},
                "UPDATED_AT": {"format": "epoch_millis", "type": "date"},
                "CREATED_AT": {"format": "epoch_millis", "type": "date"},
                "TM_TASKNAME": {"ignore_above": 2048, "type": "keyword"},
                "TM_START_TIME": {"format": "epoch_millis", "type": "date"},
                "TM_TASK_STATUS": {"ignore_above": 2048, "type": "keyword"},
                "type": {"ignore_above": 2048, "type": "keyword"},
                "timestamp": {"format": "epoch_millis", "type": "date"},
            }

        }

    }


In [ ]:
import importlib
importlib.reload(osearch)

In [ ]:
osearch.send_os(docs, index_name, schema, secretpath, yesterday_epoch)

In [ ]:
# Add a single doc to es everyday to check if pipeline is running successfully.
# This is need because we did not have rule that expires everyday
# Remember to filter it out in grafana (For example `NOT ID:00000000000000000` in lucene query)
day = start_datetime
monitoring_docs = []
while day < end_datetime:
    milisec = int(day.timestamp())*1000
    doc = {
        "ID": '00000000000000000',
        "ACCOUNT": 'cmscrab',
        "NAME": '/Pipeline/Monitoring/AOD',
        "STATE": 'P',
        "DAYS": -1,
        "EXPIRES_AT": milisec,
        "UPDATED_AT": milisec,
        "CREATED_AT": milisec,
        "TM_TASKNAME": '240000_000000:cmscrab_crab_20240000_000000',
        "TM_START_TIME": milisec,
        "TM_TASK_STATUS": 'PLACEHOLDER',
        "type": 'tape_recall_history',
        "timestamp": milisec,

    }
    monitoring_docs.append(doc)
    day += timedelta(days=1)
osearch.send_os(monitoring_docs, index_name, schema, secretpath, yesterday_epoch)

In [ ]:
# Useful query to get only the rules that gave
#query = f"""\
#repeated_ids AS (
#    SELECT ID
#    FROM rules_history
#    GROUP BY ID
#    HAVING COUNT(*) > 2
#),
#tba_t AS (
#SELECT *
#FROM rules_history
#)
#SELECT * FROM tba_t
#"""
#
#testdf = spark.sql(query)
#testdf.show(100, False)
#
# rule 37fc where latest UPDATED_AT is 43 days after the first OK state
#spark.sql("""\
#SELECT * FROM rules_history
#WHERE ID = '37fcada73f14439b88558ef792e10276'
#""").show(10, False)